# gpstextbook homework 5.4

In [1]:
import pandas as pd
import numpy as np

%pylab inline
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml" # also try "html5" (requires ffmpeg)

Populating the interactive namespace from numpy and matplotlib


In [64]:
frequency_L1 = 1575.42e6  # in Hz
frequency_L2 = 1227.60e6  # in Hz
avg_earth_radius = 6.3781e6  # in m
c0 = 299792458.0  # in m/s
omega_e_dot = 7.2921151467e-5  # in rad/s
wgs_mu = 3.986005e14  # m3 / s2

In [65]:
def read_rinex(path_rinex_file):
    return pd.read_csv(path_rinex_file, engine='python', delimiter='\s+', header=None, 
                       names=['gps_week', 'rx_week', 'prn', 'pseudoL1', 'cyclesL1', 'cyclesL2', 'pseudoP1', 
                      'pseudoP2', 'dopplerL1', 'dopplerL2'])

def read_rinex_nav(path_rinex_file):
    return pd.read_csv(path_rinex_file, engine='python', delimiter='\s+', header=None, 
                       names=['prn', 'm0', 'dn', 'e', 'sqrta', 'omg0', 'i0', 'w', 'odot', 'idot', 'cuc', 'cus', 'crc',
                      'crs', 'cic', 'cis', 'toe', 'iode', 'gps_week', 'toc', 'af0', 'af1', 'af2', 'wdot'])



In [17]:
def iono_group_delay(pseudorange_L1, pseudorange_L2, frequency_L1=frequency_L1, frequency_L2=frequency_L2):
    """This is in units of a distance (m)"""
    factor = pow(frequency_L2, 2) / (pow(frequency_L1, 2) - pow(frequency_L2, 2))
    
    def get_iono_group_delay(pseudorange_L1, pseudorange_L2):
        return factor * (pseudorange_L2 - pseudorange_L1)
    
    return get_iono_group_delay

In [53]:
def iono_obliquity_factor(zeta, avg_earth_radius=avg_earth_radius, h1=350e3):
    return pow(1. - pow((avg_earth_radius * sin(np.deg2rad(zeta))) / (avg_earth_radius + h1), 2), -0.5)
    

In [72]:
path_rinex_file = '/Users/guillaume/Workspace/gnss/data/Data/Pigeon_Point/September_18_2000/Parsed/pp091800.rio'
path_rinex_nav_file = '/Users/guillaume/Workspace/gnss/data/Data/Pigeon_Point/September_18_2000/Parsed/pp091800.rin'

df = read_rinex(path_rinex_file)
df_nav = read_rinex_nav(path_rinex_nav_file)

df['rx_week'].max()


172812.946

In [75]:
df_nav.head(3)

,prn,m0,dn,e,sqrta,omg0,i0,w,odot,idot,...,cic,cis,toe,iode,gps_week,toc,af0,af1,af2,wdot
0,1.0,-2.146908,4.809843e-09,0.004915,5153.665585,1.941131,0.961449,-1.732177,-8.235343e-09,3.089414e-10,...,3.539026e-08,7.078052e-08,86400.0,208.0,1080.0,86413.0,0.000145,1.591616e-12,0.0,0.0
1,2.0,1.240130,5.111999e-09,0.020400,5153.691751,-2.340465,0.933849,-2.091281,-8.546427e-09,-1.092903e-10,...,-4.097819e-07,4.656613e-08,86400.0,155.0,1080.0,86413.0,-0.000271,-5.229595e-12,0.0,0.0
2,3.0,-0.439754,5.334151e-09,0.001295,5153.677357,-1.259013,0.939509,0.839574,-8.711434e-09,4.714482e-11,...,-6.332994e-08,1.676381e-08,86400.0,5.0,1080.0,86413.0,0.000041,3.410605e-12,0.0,0.0


In [58]:
get_iono_group_delay = iono_group_delay(None, None)

df['slant_iono_group_delay'] = df.apply(lambda x: get_iono_group_delay(x['pseudoL1'], x['pseudoP2']), axis=1)

# approximate signal transmission time, cf homework 4.11
df['tx_time'] = df['rx_week'] - df['pseudoL1'] / c0

In [76]:
df.head(3)

,gps_week,rx_week,prn,pseudoL1,cyclesL1,cyclesL2,pseudoP1,pseudoP2,dopplerL1,dopplerL2
0,1080.0,86443.0,3.0,2.380454e+07,-8.548223e+06,-6.619810e+06,2.380454e+07,2.380456e+07,2388.252,1860.972
1,1080.0,86443.0,31.0,2.412789e+07,-3.735495e+06,-1.187441e+06,2.412789e+07,2.412791e+07,3899.132,3038.264
2,1080.0,86443.0,20.0,2.352962e+07,-3.212267e+07,-2.446205e+07,2.352962e+07,2.352964e+07,-2686.007,-2092.975


In [ ]:
def kepler(inbigM, inecc):
    """Solve Kepler's Equation
    Args:
        inbigM (array): input Mean anomaly
        inecc (array): eccentricity
    Returns:
        array: eccentric anomaly
    
    """
    
    Marr = inbigM  # protect inputs; necessary?
    eccarr = inecc
    conv = 1.0e-12  # convergence criterion
    k = 0.85

    Earr = Marr + np.sign(np.sin(Marr)) * k * eccarr  # first guess at E
    # fiarr should go to zero when converges
    fiarr = ( Earr - eccarr * np.sin(Earr) - Marr)  
    convd = np.where(np.abs(fiarr) > conv)[0]  # which indices have not converged
    nd = len(convd)  # number of unconverged elements
    count = 0

    while nd > 0:  # while unconverged elements exist
        count += 1
        
        M = Marr[convd]  # just the unconverged elements ...
        ecc = eccarr[convd]
        E = Earr[convd]

        fi = fiarr[convd]  # fi = E - e*np.sin(E)-M    ; should go to 0
        fip = 1 - ecc * np.cos(E)  # d/dE(fi) ;i.e.,  fi^(prime)
        fipp = ecc * np.sin(E)  # d/dE(d/dE(fi)) ;i.e.,  fi^(\prime\prime)
        fippp = 1 - fip  # d/dE(d/dE(d/dE(fi))) ;i.e.,  fi^(\prime\prime\prime)

        # first, second, and third order corrections to E
        d1 = -fi / fip 
        d2 = -fi / (fip + d1 * fipp / 2.0)
        d3 = -fi / (fip + d2 * fipp / 2.0 + d2 * d2 * fippp / 6.0)
        E = E + d3
        Earr[convd] = E
        fiarr = ( Earr - eccarr * np.sin( Earr ) - Marr) # how well did we do?
        convd = np.abs(fiarr) > conv  # test for convergence
        nd = np.sum(convd is True)
        
    if Earr.size > 1: 
        return Earr
    else: 
        return Earr[0]


In [ ]:
# ECEF = Earth-Centered Earth-Fixed
def from_orbital_to_ecef(x_prime, y_prime, i_k, omega_k):
    x_k = x_prime * cos(omega_k) - y_prime * cos(i_k) * sin(omega_k)
    y_k = x_prime * sin(omega_k) + y_prime * cos(i_k) * cos(omega_k)
    z_k = y_prime * sin(i_k)
    
    return np.array([x_k, y_k, z_k]).transpose()
    
def omega_k(omega_0, omega_dot, t_k, t_oe):
    return omega_0 + (omega_dot - omega_e_dot) * t_k - omega_e_dot * t_oe

def x_prime(r_k, u_k):
    return r_k * cos(u_k)

def y_prime(r_k, u_k):
    return r_k * sin(u_k)

def i_k(i_0, d_ik, i_dot, t_k)
    return i_0 + d_ik + i_dot * t_k

def r_k(A, e, Ek, d_rk):
    return A * (1. - e * cos(Ek)) + d_rk

def uk(phi_k, d_uk):
    return phi_k + d_uk

def d_ik(cis, cic, phi_k):
    return cis * sin(2.*phi_k) + cic * cos(2.*phi_k)

def d_rk(crs, crc, phi_k):
    return crs * sin(2.*phi_k) + crc * cos(2.*phi_k)

def d_uk(cus, cuc, phi_k):
    return cus * sin(2.*phi_k) + cuc * cos(2.*phi_k)

def phi_k(nu_k, omega):
    return nu_k + omega

# def Ek(e, nu_k):
#     return arccos((e+cos(nu_k))/(1.+e*cos(nu_k)))

def n(n0, dn):
    return n0 + dn

def Mk(M0, n, tk):
    return M0 + n * tk

def Ek(Mk, e):
    return kepler(Mk, e)

def nu_k(e, Ek):
    return arctan( ( ( sqrt(1.-e*e)*sin(Ek) ) / (1.-e*cos(Ek)) ) / ( (cos(Ek)-e)/(1.-e*cos(Ek)) ) )

def A(sqrta):
    return pow(sqrta , 2.)

def n0(a):
    return sqrt(wgs_mu / pow(a, 3.))

# need to solve for Ek from Mk
    
def get_ecef_coordinates_from_one_navigation_datum(nav_datum, tx_time):
    t_k = tx_time
    
    omega_k = omega_k(nav_datum['omg0'], nav_datum['odot'], t_k, nav_datum['toe'])
    
    A = A(nav_datum['sqrta'])
    n0 = n0(A)
    n = n(n0, nav_datum['dn'])
    
    Mk = Mk(nav_datum['m0'], n, t_k)
    
    Ek = Ek(Mk, nav_datum['e'])
    
    nu_k = nu_k(nav_datum['e'], Ek)
    
    phi_k = phi_k(nu_k, nav_datum['w'])
    
    d_uk = d_uk(nav_datum['cus'], nav_datum['cuc'], phi_k)
    d_rk = d_rk(nav_datum['crs'], nav_datum['crc'], phi_k)
    d_ik = d_ik(nav_datum['cis'], nav_datum['cic'], phi_k)
    
    uk = uk(phi_k, d_uk)
    r_k = r_k(A, nav_datum['e'], Ek, d_rk)
    
    x_prime = x_prime(r_k, uk)
    y_prime = y_prime(r_k, uk)
    
    d_ik = d_ik(nav_datum['cis'], nav_datum['cic'], phi_k)
    i_k = i_k(nav_datum['i0'], d_ik, nav_datum['idot'], t_k)
    
    return from_orbital_to_ecef(x_prime, y_prime, i_k, omega_k)
    

def get_satellite_pos(prn, tx_time):
    #get the nav_datum for that particular satellite in the rinex file, with the closest toe, with toe < tx_time
    
df.apply(lambda x: get_satellite_pos(x['prn'], x['tx_time']))

In [ ]:
# Re. satellite position:
# p98 of pdf IS-GPS
# homework 4.11 for receiver time -> transmission time. And yes, the difference between transmission and pseudorange/c is small but still accounts for 100s of m. of satellite movement, so worth a few cm


# in the data we have (M, e, a) on one hand, and (i, omg0, w) on the other hand 
# ? check those values might be at beginning of the week
# we transform them to E, e, a to get r_O (eq. 4.12) in the orbital coordinate system
# we transform r_O into the inertial coordinate system, noted r_I, using (i, omg0, w) cf. (4.19)
# we rotate r_I into r_T (eq. 4.21) ==> where is theta?!!!
# we know the coordinates of Pigeon Point, that is, the vector r_ref
# angle between r_T and r_ref is zenith angle

In [ ]:
df['zenith_iono_delay'] = df.apply(lambda x: x['slant_iono_group_delay'] / iono_obliquity_factor(x['zenith_angle']), 
                                   axis=1)